In [1]:
import numpy as np

In [2]:
class Layer:
    def __init__(self, activation_func, size, layer_type,weights=None,local_fields = None,after_activations=None ):
        self.activation_func = activation_func #softmax, relu
        self.size =size #number of neurons
        self.layer_type = layer_type #input node, hidden node or output node  
        self.weights = weights 
        self.local_fields=local_fields
        self.after_activations = after_activations
    def compute_outputs(self,incoming):
        self.local_fields = np.sum(incoming,axis=0) #??????
        self.after_activations = self.activation_func(self.local_fields)
        


In [3]:
class Sequential_Model: 
    def __init__(self, input_size, learning_rate,layer_list = None):
        self.input_size = input_size
        self.layer_list = layer_list #input node, hidden node or output node   
        self.learning_rate = learning_rate
        
    def add(self,layer):
        self.layer_list.append(layer)
    def fit(self,trainx,trainy):
        



IndentationError: expected an indented block (Temp/ipykernel_6156/1923386153.py, line 11)